## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

June 23 June 29


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,June-24-2025,Condo/Co-op,2000 S Ocean Blvd Unit 14b,Pompano Beach,FL,33062.0,520000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2000-S...,Beaches MLS,RX-11078315,N,Y,26.205295,-80.094233


In [8]:
print('Input county name:')
county = input()

Input county name:


 Broward


In [9]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [11]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-06-27    26
2025-06-26    17
2025-06-24    13
2025-06-25     9
Name: count, dtype: int64

In [13]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [14]:
df_top_ten = df2.head(10)

In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [15]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }
        response = requests.get(base_url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()

        # Safely drill down, falling back to {} if any level is None or missing
        info = data.get('data') or {}
        panel = info.get('mainHouseInfoPanelInfo') or {}
        house = panel.get('mainHouseInfo') or {}

        # Extract listing agents
        listing_agents = house.get('listingAgents') or []
        list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName') if len(listing_agents) > 0 else None
        list_broker_name_1 = listing_agents[0].get('brokerName')           if len(listing_agents) > 0 else None
        list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName') if len(listing_agents) > 1 else None
        list_broker_name_2 = listing_agents[1].get('brokerName')           if len(listing_agents) > 1 else None

        # Extract buying agents
        buying_agents = house.get('buyingAgents') or []
        buy_agent_name_1  = buying_agents[0].get('agentInfo', {}).get('agentName') if len(buying_agents) > 0 else None
        buy_broker_name_1 = buying_agents[0].get('brokerName')             if len(buying_agents) > 0 else None
        buy_agent_name_2  = buying_agents[1].get('agentInfo', {}).get('agentName') if len(buying_agents) > 1 else None
        buy_broker_name_2 = buying_agents[1].get('brokerName')             if len(buying_agents) > 1 else None

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return { key: None for key in [
            'url', 'list_agent_name_1','list_broker_name_1',
            'list_agent_name_2','list_broker_name_2',
            'buy_agent_name_1','buy_broker_name_1',
            'buy_agent_name_2','buy_broker_name_2'
        ] }

In [16]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [17]:
response_df = pd.DataFrame(response_list)

In [18]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [19]:
df_top_ten = merged_df

In [20]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-06-26,Condo/Co-op,1460 S Ocean Blvd #1003,Lauderdale By The Sea,FL,33062.0,2200000.0,3.0,3.5,...,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Marc Brandt,Douglas Elliman,Jeffery Kotwica,Douglas Elliman,Gilles Rais,Coldwell Banker Realty,None,None
1,PAST SALE,2025-06-24,Condo/Co-op,715 Bayshore Dr #504,Fort Lauderdale,FL,33304.0,1100000.0,3.0,3.5,...,-80.108524,https://www.redfin.com/FL/Fort-Lauderdale/715-...,Alicia Levitan,Keller Williams Realty Profess,None,None,Martine Dreyfus,EXP Realty LLC,None,None
2,PAST SALE,2025-06-27,Condo/Co-op,333 Las Olas Way #2301,Fort Lauderdale,FL,33301.0,815000.0,2.0,2.0,...,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Bobbi Ocean,Galleria International Realty,None,None,Tatiana Silva,Optimar International Realty,None,None
3,PAST SALE,2025-06-25,Condo/Co-op,3900 Galt Ocean Dr #412,Fort Lauderdale,FL,33308.0,800000.0,2.0,2.5,...,-80.097707,https://www.redfin.com/FL/Fort-Lauderdale/3900...,Roderick Perkins,Luxury Real Estate Group LLC,None,None,Megan Romine,Douglas Elliman,None,None
4,PAST SALE,2025-06-25,Condo/Co-op,3020 NE 32nd Ave #1012,Fort Lauderdale,FL,33308.0,775000.0,2.0,2.0,...,-80.103219,https://www.redfin.com/FL/Fort-Lauderdale/3020...,Eric Miller,EXP Realty LLC,None,None,Lori Novello Cortese,EXP Realty LLC,None,None
5,PAST SALE,2025-06-25,Condo/Co-op,2600 E Hallandale Beach Blvd Unit T1401,Hallandale Beach,FL,33009.0,760000.0,3.0,3.0,...,-80.122748,https://www.redfin.com/FL/Hallandale-Beach/260...,Riad Fayad,Sigma Real Estate & Investment,None,None,Debora Pludwinski,EXP Realty LLC,None,None
6,PAST SALE,2025-06-26,Condo/Co-op,1880 S Ocean Dr Unit TS401,Hallandale Beach,FL,33009.0,630000.0,3.0,3.0,...,-80.119329,https://www.redfin.com/FL/Hallandale-Beach/188...,Vera Aster,Beachfront Realty Inc,None,None,Vera Aster,Beachfront Realty Inc,None,None
7,PAST SALE,2025-06-25,Condo/Co-op,3200 Port Royale Dr N #2008,Fort Lauderdale,FL,33308.0,600000.0,2.0,2.0,...,-80.101623,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Danny Shears,Coldwell Banker Realty,None,None,Eric Miller,EXP Realty LLC,None,None
8,PAST SALE,2025-06-26,Condo/Co-op,3725 S Ocean Dr #910,Hollywood,FL,33019.0,548000.0,2.0,2.0,...,-80.118043,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,Leo Meyerkov,"Related ISG Realty, LLC.",None,None,Raul Alvarez,"My Realty Group, LLC.",None,None
9,PAST SALE,2025-06-24,Condo/Co-op,2000 S Ocean Blvd Unit 14b,Pompano Beach,FL,33062.0,520000.0,2.0,2.0,...,-80.094233,https://www.redfin.com/FL/Pompano-Beach/2000-S...,Nick Wulf,REDFIN CORPORATION,None,None,Randy Lloyd,LoKation,None,None


## Current Week's Values

In [21]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

65


In [22]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$326,772


In [23]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$264


In [24]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$21,240,150


In [25]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           65              326772          264                     21240150.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [26]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
120
-----------
Input Previous Week Condo Average Sales Price:
293057
-----------
Input Previous Week Condo Average PSF:
252
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
35166873.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [27]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [28]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [29]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [30]:
pd.set_option('display.max_columns',None)

In [31]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-26,Condo/Co-op,1460 S Ocean Blvd #1003,Lauderdale By The Sea,FL,33062.0,2200000.0,3.0,3.5,Europa By The Sea,3523.0,NaN,2006.0,NaN,624.0,3859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Beaches MLS,F10476660,N,Y,26.213195,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Marc Brandt,Douglas Elliman,Jeffery Kotwica,Douglas Elliman,Gilles Rais,Coldwell Banker Realty,None,None,1,orange


In [32]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [33]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-26,Condo/Co-op,1460 S Ocean Blvd #1003,Lauderdale By The Sea,FL,33062.0,2200000.0,3.0,3.5,Europa By The Sea,3523.0,NaN,2006.0,NaN,624.0,3859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Beaches MLS,F10476660,N,Y,26.213195,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Marc Brandt,Douglas Elliman,Jeffery Kotwica,Douglas Elliman,Gilles Rais,Coldwell Banker Realty,None,None,1,orange
1,PAST SALE,2025-06-24,Condo/Co-op,715 Bayshore Dr #504,Fort Lauderdale,FL,33304.0,1100000.0,3.0,3.5,La Rive,2680.0,NaN,2004.0,NaN,410.0,2791.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/715-...,Beaches MLS,F10498085,N,Y,26.133158,-80.108524,https://www.redfin.com/FL/Fort-Lauderdale/715-...,Alicia Levitan,Keller Williams Realty Profess,None,None,Martine Dreyfus,EXP Realty LLC,None,None,2,blue
2,PAST SALE,2025-06-27,Condo/Co-op,333 Las Olas Way #2301,Fort Lauderdale,FL,33301.0,815000.0,2.0,2.0,LAS OLAS RIVER HOUSE COND,1161.0,NaN,2004.0,NaN,702.0,1503.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,MARMLS,A11679228,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Bobbi Ocean,Galleria International Realty,None,None,Tatiana Silva,Optimar International Realty,None,None,3,blue
3,PAST SALE,2025-06-25,Condo/Co-op,3900 Galt Ocean Dr #412,Fort Lauderdale,FL,33308.0,800000.0,2.0,2.5,Playa Del Mar,1649.0,NaN,1975.0,NaN,485.0,1109.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3900...,Beaches MLS,F10485618,N,Y,26.175849,-80.097707,https://www.redfin.com/FL/Fort-Lauderdale/3900...,Roderick Perkins,Luxury Real Estate Group LLC,None,None,Megan Romine,Douglas Elliman,None,None,4,blue
4,PAST SALE,2025-06-25,Condo/Co-op,3020 NE 32nd Ave #1012,Fort Lauderdale,FL,33308.0,775000.0,2.0,2.0,Tides At Bridgeside Sq,1600.0,NaN,2002.0,NaN,484.0,1892.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3020...,Beaches MLS,F10483468,N,Y,26.166161,-80.103219,https://www.redfin.com/FL/Fort-Lauderdale/3020...,Eric Miller,EXP Realty LLC,None,None,Lori Novello Cortese,EXP Realty LLC,None,None,5,blue
5,PAST SALE,2025-06-25,Condo/Co-op,2600 E Hallandale Beach Blvd Unit T1401,Hallandale Beach,FL,33009.0,760000.0,3.0,3.0,BEACHWALK CONDOMINIUM,1737.0,NaN,2015.0,NaN,438.0,2100.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/260...,MARMLS,A11592428,N,Y,25.985939,-80.122748,https://www.redfin.com/FL/Hallandale-Beach/260...,Riad Fayad,Sigma Real Estate & Investment,None,None,Debora Pludwinski,EXP Realty LLC,None,None,6,blue
6,PAST SALE,2025-06-26,Condo/Co-op,1880 S Ocean Dr Unit TS401,Hallandale Beach,FL,33009.0,630000.0,3.0,3.0,LA MER ESTATES WEST CONDO,1940.0,NaN,1974.0,NaN,325.0,1731.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/188...,MARMLS,A11782835,N,Y,25.982872,-80.119329,https://www.redfin.com/FL/Hallandale-Beach/188...,Vera Aster,Beachfront Realty Inc,None,None,Vera Aster,Beachfront Realty Inc,None,None,7,blue
7,PAST SALE,2025-06-25,Condo/Co-op,3200 Port Royale Dr N #2008,Fort Lauderdale,FL,33308.0,600000.0,2.0,2.0,TOWER PORTSMOUTH AT PORT,1365.0,NaN,1981.0,NaN,440.0,1278.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,MARMLS,A11757014,N,Y,26.210739,-80.101623,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Danny Shears,Coldwell Banker Realty,None,None,Eric Miller,EXP Realty LLC,None,None,8,blue
8,PAST SALE,2025-06-26,Condo/Co-op,3725 S Ocean Dr #910,Hollywood,FL,33019.0,548000.0,2.0,2.0,SEA AIR TOWERS CONDO,1431.0,NaN,1971.0,NaN,383.0,995.0,Sold,NaN,NaN,https://www.redfin.com

In [77]:
%store -r map_box_api_key

In [78]:
map_box_api_key

'pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ'

In [35]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=9,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [36]:
m.save('index.html')

## Data snagger

In [37]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [38]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-26,Condo/Co-op,1460 S Ocean Blvd #1003,Lauderdale By The Sea,FL,33062.0,2200000.0,3.0,3.5,Europa By The Sea,3523.0,NaN,2006.0,NaN,624.0,3859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Beaches MLS,F10476660,N,Y,26.213195,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Marc Brandt,Douglas Elliman,Jeffery Kotwica,Douglas Elliman,Gilles Rais,Coldwell Banker Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [39]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_06302025


# CREATE TEMPLATE 

In [40]:
muni_set = set(df_top_ten['CITY'])

In [41]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [42]:
df_top_ten.reset_index(inplace=True,drop=True)

In [43]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [44]:
top_sale

'Europa By The Sea at 1460 S Ocean Blvd #1003 in Lauderdale By The Sea'

In [45]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [46]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-26,Condo/Co-op,1460 S Ocean Blvd #1003,Lauderdale By The Sea,FL,33062.0,2200000.0,3.0,3.5,Europa By The Sea,3523.0,NaN,2006.0,NaN,624.0,3859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Beaches MLS,F10476660,N,Y,26.213195,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Marc Brandt,Douglas Elliman,Jeffery Kotwica,Douglas Elliman,Gilles Rais,Coldwell Banker Realty,None,None,1,orange,Europa By The Sea at 1460 S Ocean Blvd #1003 i...
1,PAST SALE,2025-06-24,Condo/Co-op,715 Bayshore Dr #504,Fort Lauderdale,FL,33304.0,1100000.0,3.0,3.5,La Rive,2680.0,NaN,2004.0,NaN,410.0,2791.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/715-...,Beaches MLS,F10498085,N,Y,26.133158,-80.108524,https://www.redfin.com/FL/Fort-Lauderdale/715-...,Alicia Levitan,Keller Williams Realty Profess,None,None,Martine Dreyfus,EXP Realty LLC,None,None,2,blue,La Rive at 715 Bayshore Dr #504 in Fort Lauder...
2,PAST SALE,2025-06-27,Condo/Co-op,333 Las Olas Way #2301,Fort Lauderdale,FL,33301.0,815000.0,2.0,2.0,LAS OLAS RIVER HOUSE COND,1161.0,NaN,2004.0,NaN,702.0,1503.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,MARMLS,A11679228,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Bobbi Ocean,Galleria International Realty,None,None,Tatiana Silva,Optimar International Realty,None,None,3,blue,LAS OLAS RIVER HOUSE COND at 333 Las Olas Way ...
3,PAST SALE,2025-06-25,Condo/Co-op,3900 Galt Ocean Dr #412,Fort Lauderdale,FL,33308.0,800000.0,2.0,2.5,Playa Del Mar,1649.0,NaN,1975.0,NaN,485.0,1109.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3900...,Beaches MLS,F10485618,N,Y,26.175849,-80.097707,https://www.redfin.com/FL/Fort-Lauderdale/3900...,Roderick Perkins,Luxury Real Estate Group LLC,None,None,Megan Romine,Douglas Elliman,None,None,4,blue,Playa Del Mar at 3900 Galt Ocean Dr #412 in Fo...
4,PAST SALE,2025-06-25,Condo/Co-op,3020 NE 32nd Ave #1012,Fort Lauderdale,FL,33308.0,775000.0,2.0,2.0,Tides At Bridgeside Sq,1600.0,NaN,2002.0,NaN,484.0,1892.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3020...,Beaches MLS,F10483468,N,Y,26.166161,-80.103219,https://www.redfin.com/FL/Fort-Lauderdale/3020...,Eric Miller,EXP Realty LLC,None,None,Lori Novello Cortese,EXP Realty LLC,None,None,5,blue,Tides At Bridgeside Sq at 3020 NE 32nd Ave #10...
5,PAST SALE,2025-06-25,Condo/Co-op,2600 E Hallandale Beach Blvd Unit T1401,Hallandale Beach,FL,33009.0,760000.0,3.0,3.0,BEACHWALK CONDOMINIUM,1737.0,NaN,2015.0,NaN,438.0,2100.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/260...,MARMLS,A11592428,N,Y,25.985939,-80.122748,https://www.redfin.com/FL/Hallandale-Beach/260...,Riad Fayad,Sigma Real Estate & Investment,None,None,Debora Pludwinski,EXP Realty LLC,None,None,6,blue,BEACHWALK CONDOMINIUM at 2600 E Hallandale Bea...
6,PAST SALE,2025-06-26,Condo/Co-op,1880 S Ocean Dr Unit TS401,Hallandale Beach,FL,33009.0,630000.0,3.0,3.0,LA MER ESTATES WEST CONDO,1940.0,NaN,1974.0,NaN,325.0,1731.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/188...,MARMLS,A11782835,N,Y,25.982872,-80.119329,https://www.redfin.com/FL/Hallandale-Beach/188...,Vera Aster,Beachfront Realty Inc,None,None,Vera Aster,Beachfront Realty Inc,None,None,7,blue,LA MER ESTATES WEST CONDO at 1880 S Ocean Dr U...
7,PAST SALE,2025-06-25,Condo/Co-op,3200 Port Royale Dr N #2008,Fort Lauderdale,FL,33308.0,600000.0,2.0,2.0,TOWER PORTSMOUTH AT PORT,1365.0,NaN,1981.0,NaN,440.0,1278.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,MARMLS

In [48]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Europa By The Sea closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $520,000 to $2,200,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hollywood, Pompano Beach, Fort Lauderdale, Lauderdale By The Sea, Hallandale Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 65 condo sales totaling $21,240,150 million from June 23 to June 29. The previous week, brokers closed 120 condo sales totaling $35,166,873.

Last week’s units sold for an average of $326,772, higher than the $293,057 average price from

In [49]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [50]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [51]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [52]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-26,Condo/Co-op,1460 S Ocean Blvd #1003,Lauderdale By The Sea,FL,33062.0,2200000.0,3.0,3.5,Europa By The Sea,3523.0,NaN,2006.0,NaN,624.0,3859.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Beaches MLS,F10476660,N,Y,26.213195,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Marc Brandt,Douglas Elliman,Jeffery Kotwica,Douglas Elliman,Gilles Rais,Coldwell Banker Realty,None,None,1,orange,Europa By The Sea at 1460 S Ocean Blvd #1003 i...
1,PAST SALE,2025-06-24,Condo/Co-op,715 Bayshore Dr #504,Fort Lauderdale,FL,33304.0,1100000.0,3.0,3.5,La Rive,2680.0,NaN,2004.0,NaN,410.0,2791.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/715-...,Beaches MLS,F10498085,N,Y,26.133158,-80.108524,https://www.redfin.com/FL/Fort-Lauderdale/715-...,Alicia Levitan,Keller Williams Realty Profess,None,None,Martine Dreyfus,EXP Realty LLC,None,None,2,blue,La Rive at 715 Bayshore Dr #504 in Fort Lauder...
2,PAST SALE,2025-06-27,Condo/Co-op,333 Las Olas Way #2301,Fort Lauderdale,FL,33301.0,815000.0,2.0,2.0,LAS OLAS RIVER HOUSE COND,1161.0,NaN,2004.0,NaN,702.0,1503.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/333-...,MARMLS,A11679228,N,Y,26.118354,-80.141518,https://www.redfin.com/FL/Fort-Lauderdale/333-...,Bobbi Ocean,Galleria International Realty,None,None,Tatiana Silva,Optimar International Realty,None,None,3,blue,LAS OLAS RIVER HOUSE COND at 333 Las Olas Way ...
3,PAST SALE,2025-06-25,Condo/Co-op,3900 Galt Ocean Dr #412,Fort Lauderdale,FL,33308.0,800000.0,2.0,2.5,Playa Del Mar,1649.0,NaN,1975.0,NaN,485.0,1109.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3900...,Beaches MLS,F10485618,N,Y,26.175849,-80.097707,https://www.redfin.com/FL/Fort-Lauderdale/3900...,Roderick Perkins,Luxury Real Estate Group LLC,None,None,Megan Romine,Douglas Elliman,None,None,4,blue,Playa Del Mar at 3900 Galt Ocean Dr #412 in Fo...
4,PAST SALE,2025-06-25,Condo/Co-op,3020 NE 32nd Ave #1012,Fort Lauderdale,FL,33308.0,775000.0,2.0,2.0,Tides At Bridgeside Sq,1600.0,NaN,2002.0,NaN,484.0,1892.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3020...,Beaches MLS,F10483468,N,Y,26.166161,-80.103219,https://www.redfin.com/FL/Fort-Lauderdale/3020...,Eric Miller,EXP Realty LLC,None,None,Lori Novello Cortese,EXP Realty LLC,None,None,5,blue,Tides At Bridgeside Sq at 3020 NE 32nd Ave #10...
5,PAST SALE,2025-06-25,Condo/Co-op,2600 E Hallandale Beach Blvd Unit T1401,Hallandale Beach,FL,33009.0,760000.0,3.0,3.0,BEACHWALK CONDOMINIUM,1737.0,NaN,2015.0,NaN,438.0,2100.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/260...,MARMLS,A11592428,N,Y,25.985939,-80.122748,https://www.redfin.com/FL/Hallandale-Beach/260...,Riad Fayad,Sigma Real Estate & Investment,None,None,Debora Pludwinski,EXP Realty LLC,None,None,6,blue,BEACHWALK CONDOMINIUM at 2600 E Hallandale Bea...
6,PAST SALE,2025-06-26,Condo/Co-op,1880 S Ocean Dr Unit TS401,Hallandale Beach,FL,33009.0,630000.0,3.0,3.0,LA MER ESTATES WEST CONDO,1940.0,NaN,1974.0,NaN,325.0,1731.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/188...,MARMLS,A11782835,N,Y,25.982872,-80.119329,https://www.redfin.com/FL/Hallandale-Beach/188...,Vera Aster,Beachfront Realty Inc,None,None,Vera Aster,Beachfront Realty Inc,None,None,7,blue,LA MER ESTATES WEST CONDO at 1880 S Ocean Dr U...
7,PAST SALE,2025-06-25,Condo/Co-op,3200 Port Royale Dr N #2008,Fort Lauderdale,FL,33308.0,600000.0,2.0,2.0,TOWER PORTSMOUTH AT PORT,1365.0,NaN,1981.0,NaN,440.0,1278.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,MARMLS

In [53]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Europa By The Sea closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $520,000 to $2,200,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hollywood, Pompano Beach, Fort Lauderdale, Lauderdale By The Sea, Hallandale Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 65 condo sales totaling $21,240,150 million from June 23 to June 29. The previous week, brokers closed 120 condo sales totaling $35,166,873.

Last week’s units sold for an average of $326,772, higher than the $293,057 average price from

In [54]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Pompano-Beach/1460-S-Ocean-Blvd-33062/unit-1003/home/42055652


In [55]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/715-Bayshore-Dr-33304/unit-504/home/42028874


In [56]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Pompano-Beach/2000-S-Ocean-Blvd-33062/unit-14B/home/41698871


In [57]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Hollywood/3725-S-Ocean-Dr-33019/unit-910/home/41970568


In [58]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/333-Las-Olas-Way-33301/unit-2301/home/42026546


In [59]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Hallandale-Beach/1880-N-Ocean-Dr-33009/unit-TS401/home/41957060


In [76]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-16,Condo/Co-op,3100 N Ocean Blvd #2201,Fort Lauderdale,FL,33308.0,1700000.0,3.0,3.0,L'hermitage,2318.0,NaN,1997.0,NaN,733.0,3119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10495199,N,Y,26.168258,-80.099279,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Patricia Post,"Post and Post, Inc.",None,None,Patricia Post,"Post and Post, Inc.",None,None,1,orange,L'hermitage at 3100 N Ocean Blvd #2201 in Fort...
1,PAST SALE,2025-06-16,Condo/Co-op,3055 Harbor Dr #603,Fort Lauderdale,FL,33316.0,1600000.0,3.0,3.5,Harborage Place,3368.0,NaN,1998.0,NaN,475.0,5188.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3055...,Beaches MLS,F10475427,N,Y,26.109135,-80.108540,https://www.redfin.com/FL/Fort-Lauderdale/3055...,Elizabeth Caldwell,"Compass Florida, LLC",Rebecca Baughman,"Compass Florida, LLC",Susan Rindley,One Sotheby's Int'l Realty,None,None,2,blue,Harborage Place at 3055 Harbor Dr #603 in Fort...
2,PAST SALE,2025-06-20,Condo/Co-op,6100 N Ocean Dr Unit 3S,Hollywood,FL,33019.0,1600000.0,3.0,3.0,SEASIDE VILLAGE WEST COND,2180.0,NaN,2018.0,NaN,734.0,1938.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6100-N-Oce...,MARMLS,A11735935,N,Y,26.052075,-80.114644,https://www.redfin.com/FL/Hollywood/6100-N-Oce...,None,None,None,None,None,None,None,None,3,blue,SEASIDE VILLAGE WEST COND at 6100 N Ocean Dr U...
3,PAST SALE,2025-06-16,Condo/Co-op,5000 N Ocean Blvd #1402,Lauderdale By The Sea,FL,33308.0,900000.0,2.0,2.5,Sea Ranch Club,1780.0,NaN,1979.0,NaN,506.0,681.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10449724,N,Y,26.198972,-80.094812,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Angelica Pooz,Xcellence Realty Inc.,None,None,Angelica Pooz,Xcellence Realty Inc.,None,None,4,blue,Sea Ranch Club at 5000 N Ocean Blvd #1402 in L...
4,PAST SALE,2025-06-17,Condo/Co-op,4111 S Ocean Dr Unit MPH03,Hollywood,FL,33019.0,890000.0,2.0,2.0,Lyfe Resort And Residence,1136.0,NaN,2017.0,NaN,783.0,1580.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Beaches MLS,F10496568,N,Y,25.986806,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Jaime Delgado,Al dia Real Estate,None,None,Antonio Pineda,Expert Team One Realty LLC,None,None,5,blue,Lyfe Resort And Residence at 4111 S Ocean Dr U...
5,PAST SALE,2025-06-20,Condo/Co-op,350 SE 2nd St #970,Fort Lauderdale,FL,33301.0,790000.0,3.0,2.0,350 LAS OLAS PLACE CONDO,1650.0,NaN,2005.0,NaN,479.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/350-...,MARMLS,A11733963,N,Y,26.120405,-80.139762,https://www.redfin.com/FL/Fort-Lauderdale/350-...,Patty Sierra,Sierra Realty,None,None,Patty Sierra,Sierra Realty,None,None,6,blue,350 LAS OLAS PLACE CONDO at 350 SE 2nd St #970...
6,PAST SALE,2025-06-18,Condo/Co-op,1905 N Ocean Blvd Unit 6A,Fort Lauderdale,FL,33305.0,725000.0,2.0,2.0,Shore Club Condominium,1500.0,NaN,1976.0,NaN,483.0,1797.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1905...,Beaches MLS,F10431446,N,Y,26.153396,-80.101460,https://www.redfin.com/FL/Fort-Lauderdale/1905...,Romina Oliviero,Lauderdale Beach Properties,None,None,Roni Sterin PA,The Keyes Company,None,None,7,blue,Shore Club Condominium at 1905 N Ocean Blvd Un...
7,PAST SALE,2025-06-17,Condo/Co-op,4520 El Mar Dr #1,Fort Lauderdale,FL,33308.0,650000.0,1.0,2.0,Southern Seas,820.0,NaN,1957.0,NaN,793.0,1526.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10480424,N,Y,26.193553,-80.095086,https://www.redfin.com/FL/Lauderdale-by-the-S

## Time on Market Calculator

In [61]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 4, 14) ## List (Earlier) date
date2 = datetime(2025, 6, 24) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

71


## Clean CSV for Datawrapper Chart

In [62]:
chart_df = df_top_ten

In [63]:
chart_df = chart_df.fillna(" ")

In [64]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [65]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [66]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [67]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [68]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    '$/SQUARE FEET',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [69]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [70]:
# Coerce non-numeric to NaN, then you’ll have a float column
chart_df['PPSF'] = pd.to_numeric(chart_df['PPSF'], errors='coerce')

In [71]:
chart_df['SQUARE FEET'] = pd.to_numeric(chart_df['SQUARE FEET'], errors='coerce')

In [72]:
chart_df['PPSF'].astype(float)

0    624.0
1    410.0
2    702.0
3    485.0
4    484.0
5    438.0
6    325.0
7    440.0
8    383.0
9    385.0
Name: PPSF, dtype: float64

In [73]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [74]:
csv_date_string = today.strftime("%m_%d_%Y")

In [75]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [76]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-06-26,Europa By The Sea,1460 S Ocean Blvd #1003 in Lauderdale By The Sea,"$2,200,000","3,523",$624,2006,Marc Brandt Douglas Elliman Jeffery Kotwica Do...,Gilles Rais Coldwell Banker Realty
1,2025-06-24,La Rive,715 Bayshore Dr #504 in Fort Lauderdale,"$1,100,000","2,680",$410,2004,Alicia Levitan Keller Williams Realty Profess,Martine Dreyfus EXP Realty LLC
2,2025-06-27,LAS OLAS RIVER HOUSE COND,333 Las Olas Way #2301 in Fort Lauderdale,"$815,000","1,161",$702,2004,Bobbi Ocean Galleria International Realty,Tatiana Silva Optimar International Realty
3,2025-06-25,Playa Del Mar,3900 Galt Ocean Dr #412 in Fort Lauderdale,"$800,000","1,649",$485,1975,Roderick Perkins Luxury Real Estate Group LLC,Megan Romine Douglas Elliman
4,2025-06-25,Tides At Bridgeside Sq,3020 NE 32nd Ave #1012 in Fort Lauderdale,"$775,000","1,600",$484,2002,Eric Miller EXP Realty LLC,Lori Novello Cortese EXP Realty LLC
5,2025-06-25,BEACHWALK CONDOMINIUM,2600 E Hallandale Beach Blvd Unit T1401 in Hal...,"$760,000","1,737",$438,2015,Riad Fayad Sigma Real Estate & Investment,Debora Pludwinski EXP Realty LLC
6,2025-06-26,LA MER ESTATES WEST CONDO,1880 S Ocean Dr Unit TS401 in Hallandale Beach,"$630,000","1,940",$325,1974,Vera Aster Beachfront Realty Inc,Vera Aster Beachfront Realty Inc
7,2025-06-25,TOWER PORTSMOUTH AT PORT,3200 Port Royale Dr N #2008 in Fort Lauderdale,"$600,000","1,365",$440,1981,Danny Shears Coldwell Banker Realty,Eric Miller EXP Realty LLC
8,2025-06-26,SEA AIR TOWERS CONDO,3725 S Ocean Dr #910 in Hollywood,"$548,000","1,431",$383,1971,"Leo Meyerkov Related ISG Realty, LLC.","Raul Alvarez My Realty Group, LLC."
9,2025-06-24,Royal Coast Condo,2000 S Ocean Blvd Unit 14b in Pompano Beach,"$520,000","1,350",$385,1971,Nick Wulf REDFIN CORPORATION,Randy Lloyd LoKation
